In [1]:
import pandas as pd
df = pd.read_csv("data/0510-citation-pub-data.csv")

/var/folders/rw/s7c9z30x7fj_2gcngv0yhv6w0000gn/T/ipykernel_63755/177954249.py:2: DtypeWarning: Columns (12,21,22,24,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/0510-citation-pub-data.csv")


In [ ]:
set(df['School'].tolist())

{'CUHK', 'CityU', 'HKU', 'HKUST', 'NTU', 'NUS', 'SMU'}

In [11]:
df = df[df['School'].isin(['CityU', 'CUHK', 'HKU', 'HKUST'])]
df = df[['Researcher of Interest', 'School']]
df = df.drop_duplicates()

In [13]:
print(df.iloc[0,])

Researcher of Interest    Li QIU
School                     HKUST
Name: 16907, dtype: object


In [ ]:
from scholarly import scholarly

# use the Google Scholar page for a researcher to figure out which year this person first published
def get_first_publication_year(name: str):
    search_query = scholarly.search_author(name)
    author = next(search_query, None)

    if author:
        author = scholarly.fill(author)
        publications = author.get('publications', [])
        if publications:
            first_pub_year = min(pub.get('bib', {}).get('pub_year', float('inf')) for pub in publications)
            return first_pub_year if first_pub_year != float('inf') else None
    return None

In [ ]:
def get_undergraduate_institution(name: str):
    